# Задача 2.4 $QR$-разложение

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def QR_decomposition(A):  # using Modified Gram-Schmidt
    A = A.copy()
    m, n = A.shape
    Q = np.zeros([m,n], dtype=np.float64)
    R = np.zeros([n,n], dtype=np.float64)
    for i in range(n):
        R[i,i] = np.linalg.norm(A[:,i])
        Q[:,i] = A[:,i] / R[i,i]
        for j in range(i, n):
            tmp = 0.0
            for k in range(m):
                tmp += Q[k,i] * A[k,j]
            R[i,j] = tmp  # np.dot(Q[:,i], A[:,j]) на всякий случай воздержался от использования, хотя это просто скалярное произведение векторов
            A[:,j] = A[:,j] - R[i,j] * Q[:,i]  # здесь не матрично-векторное умножение, а просто умножение вектора на скаляр
    return Q, R


def LowerTriangular_solver(L, b):
	assert np.allclose(L, np.tril(L));  # checks if L is Lower Triangular 
	n = L.shape[0]
	y = np.zeros(n)
	for i in range(n):
		tmp_sum = 0
		for j in range(i):
			tmp_sum += L[i][j] * y[j]
		y[i] = (b[i] - tmp_sum) / L[i][i]
		
	return y


def UpperTriangular_solver(U, y):
	assert np.allclose(U, np.triu(U));  # checks if L is Upper Triangular 
	return LowerTriangular_solver(np.flip(np.flip(U, 1), 0), y[::-1])[::-1]


def solve(A, b):
	n = A.shape[0]
	# A = Q @ R
	Q, R = QR_decomposition(A)
	# Q @ R @ x = A @ x = b
    # R @ x = Q.T @ b = y
	y = np.dot(Q.T, b)
	x = UpperTriangular_solver(R, y)
	return Q, R, x

### Квадратная невырожденная матрица

In [2]:
n = 3
A = np.random.rand(n,n).astype(np.float64)
A

array([[0.38145371, 0.58892845, 0.11125714],
       [0.23889599, 0.23357124, 0.036153  ],
       [0.57023185, 0.96455704, 0.55217106]])

Библиотечное $QR$-разложение

In [3]:
np_Q, np_R = np.linalg.qr(A) 

In [4]:
np_Q

array([[-0.52508641, -0.07182379, -0.84801274],
       [-0.32884995, -0.90191523,  0.28001184],
       [-0.78494712,  0.42589936,  0.44996418]])

In [5]:
np_R

array([[-0.72645894, -1.14317448, -0.5037336 ],
       [ 0.        ,  0.1578437 ,  0.19457145],
       [ 0.        ,  0.        ,  0.16423299]])

Кастомное $QR$-разложение

In [6]:
my_Q, my_R = QR_decomposition(A)

In [7]:
my_Q

array([[ 0.52508641, -0.07182379, -0.84801274],
       [ 0.32884995, -0.90191523,  0.28001184],
       [ 0.78494712,  0.42589936,  0.44996418]])

In [8]:
my_R

array([[0.72645894, 1.14317448, 0.5037336 ],
       [0.        , 0.1578437 , 0.19457145],
       [0.        , 0.        , 0.16423299]])

Понятно, что нужно сравнивать матрицы на равенство с точностью до замены знаков в столбцах(это не влияет на ортогональность столбцов)

In [9]:
np.linalg.norm(abs(my_Q) - abs(np_Q))

1.1096157052992535e-15

In [10]:
np.linalg.norm(abs(my_R) - abs(np_R))

3.1155556357914184e-16

Посмотрим на решение системы $Ax = b$ методом наименьших квадратов

In [11]:
x = np.random.rand(n).astype(np.float64)
pd.DataFrame(x)

,0
0,0.835625
1,0.543177
2,0.719569


In [12]:
# Использую матричное умножение для проверки примера. Это не реализация требуемой функции
b = np.dot(A, x.T)
pd.DataFrame(b)

,0
0,0.718702
1,0.352513
2,1.397750


Библиотечное решение системы

In [13]:
np_sol = np.linalg.lstsq(A, b)[0]
pd.DataFrame(np_sol)

,0
0,0.835625
1,0.543177
2,0.719569


Кастомное решение системы

In [14]:
my_sol = solve(A, b)[2]
pd.DataFrame(my_sol)

,0
0,0.835625
1,0.543177
2,0.719569


Норма разности решений

In [15]:
np.linalg.norm(np_sol - my_sol)

1.4132140477967265e-14

### Прямоугольная матрица c m > n с линейно независимыми столбцами

In [16]:
m = 7
n = 4
A = np.random.rand(m,n).astype(np.float64)
A

array([[0.15247588, 0.59998488, 0.61891827, 0.09013963],
       [0.8521413 , 0.3647799 , 0.28021806, 0.0705466 ],
       [0.68098029, 0.52526189, 0.81476201, 0.04880329],
       [0.18791141, 0.90964386, 0.43208912, 0.79132965],
       [0.86304625, 0.45328293, 0.03452462, 0.06939149],
       [0.4145645 , 0.88916801, 0.62164732, 0.50039746],
       [0.19602952, 0.96586665, 0.38469197, 0.63597795]])

Библиотечное $QR$-разложение

In [17]:
np_Q, np_R = np.linalg.qr(A) 

In [18]:
np_Q

array([[-0.10271571,  0.3390041 , -0.4008214 ,  0.65471615],
       [-0.57404684, -0.26136386,  0.0432854 , -0.22792654],
       [-0.45874386, -0.04022565, -0.67519511, -0.17422517],
       [-0.12658693,  0.540596  ,  0.17033379, -0.55854974],
       [-0.58139299, -0.20422567,  0.50517166,  0.35991217],
       [-0.27927228,  0.38630743, -0.10094642, -0.07839352],
       [-0.13205572,  0.57616097,  0.29525913,  0.20342496]])

In [19]:
np_R

array([[-1.48444557, -1.26654202, -0.89737683, -0.4363914 ],
       [ 0.        ,  1.38609274,  0.79212895,  0.98350727],
       [ 0.        ,  0.        , -0.64419868,  0.24108176],
       [ 0.        ,  0.        ,  0.        , -0.29244259]])

Кастомное $QR$-разложение

In [20]:
my_Q, my_R = QR_decomposition(A)

In [21]:
my_Q

array([[ 0.10271571,  0.3390041 ,  0.4008214 , -0.65471615],
       [ 0.57404684, -0.26136386, -0.0432854 ,  0.22792654],
       [ 0.45874386, -0.04022565,  0.67519511,  0.17422517],
       [ 0.12658693,  0.540596  , -0.17033379,  0.55854974],
       [ 0.58139299, -0.20422567, -0.50517166, -0.35991217],
       [ 0.27927228,  0.38630743,  0.10094642,  0.07839352],
       [ 0.13205572,  0.57616097, -0.29525913, -0.20342496]])

In [22]:
my_R

array([[ 1.48444557,  1.26654202,  0.89737683,  0.4363914 ],
       [ 0.        ,  1.38609274,  0.79212895,  0.98350727],
       [ 0.        ,  0.        ,  0.64419868, -0.24108176],
       [ 0.        ,  0.        ,  0.        ,  0.29244259]])

Норма разности разложений

In [23]:
np.linalg.norm(abs(my_Q) - abs(np_Q))

1.6558533042896306e-15

In [24]:
np.linalg.norm(abs(my_R) - abs(np_R))

7.005190574351762e-16

Посмотрим на решение системы $Ax = b$ методом наименьших квадратов

In [25]:
x = np.random.rand(n).astype(np.float64)
pd.DataFrame(x)

,0
0,0.592047
1,0.736791
2,0.459491
3,0.477040


In [26]:
# Использую матричное умножение для проверки примера. Это не реализация требуемой функции
b = np.dot(A, x.T)
pd.DataFrame(b)

,0
0,0.859724
1,0.935686
2,1.187838
3,1.357506
4,0.893905
5,1.424923
6,1.307850


Библиотечное решение системы

In [27]:
np_sol = np.linalg.lstsq(A, b)[0]
pd.DataFrame(np_sol)

,0
0,0.592047
1,0.736791
2,0.459491
3,0.477040


Кастомное решение системы

In [28]:
my_sol = solve(A, b)[2]
pd.DataFrame(my_sol)

,0
0,0.592047
1,0.736791
2,0.459491
3,0.477040


Норма разности решений

In [29]:
np.linalg.norm(np_sol - my_sol)

4.828193870057736e-15